In [8]:
import geopandas as gpd
import pandas as pd
import json
import os
import pygeos
import shapely
import rasterio
import json
import pyproj
import numpy as np
from pathlib import Path
from tqdm import tqdm
#TODO: update to shapely2.0 instead of pygeos

In [2]:
### 1. Split geotiff args ###

raster_folder = Path("../NSO/NSO_big_tiles")
tile_output_folder = Path("../NSO/NSO_small_tiles")
#make output directory if does not exist
if not os.path.exists(tile_output_folder):
    os.makedirs(tile_output_folder)    
width = 500 #width of the new tiles
height = 500 #height of the new tiles
nso_pix = 0.5 #pixel size of the raster

### 2. Reproject geodataframe of a annotation shapefile args ###

annotations_crs = "epsg:4326"
raster_crs = "epsg:28992"
annotation_folder = Path("../NSO/annotations_shp")
# !!!!!! Save GIS made annotations to annotation folder!!!!!! #
annotation_file = "fixed_substations_NL_annotations.shp"
# path to the annotation shapefile 
shp_path = os.path.join(annotation_folder, annotation_file)

### 3.Spit annotations args ###

# Make folder for geojsons
json_folder = Path("../NSO/geojsons")
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

In [3]:
def split_geotiff (rasters_path, output_dir, width, height, pixel_size):
    """
    This function splits a given geotiff raster image into multiple tiles with a specified width and height.
    Also skips tiles with more than 10% of black pixels

    Parameters:
    rasters_path (str): File path to the input folder with the rasters.
    output_dir (str): File path to the output folder where the created tiles will be saved.
    width (int): Width of the tiles in pixels.
    height (int): Height of the tiles in pixels.
    pixel_size (float): Pixel size of the raster.

    Returns:
    None
    """

    files = os.listdir(rasters_path)

    # Filter for only the files that end with ".tif"
    tif_files = [file for file in files if file.endswith(".tif")]

    #raster id to be used in the output file name
    i = 0

    # Iterate through all the raster files in the input directory if ends with "tif"
    for tif_file in tqdm(tif_files):
        i+=1
        img_id = tif_file.split(".tif")[0]
        raster = os.path.join(rasters_path, tif_file)
        # Open the input raster file using rasterio.open() and read the raster image and metadata information
        with rasterio.open(raster) as src:
            out_image = src.read()
            out_meta = src.meta
            out_transform = src.transform
            
            # Array for the itermediate heights and widths of the tiles based on the specified width and height parameters
            heights = np.arange(0, out_image.shape[1], height)
            widths = np.arange(0, out_image.shape[2], width)
            
            # Extract the origin x (min) and y(max) of the raster image
            min_x = out_transform[2]
            max_y = out_transform[5]

            # Iterate through all the tiles and create a new tile image
            for x in range(len(heights)):
                for y in range(len(widths)):
                    # Skip the last value of array heights [x] and widths [y] to avoid [IndexError: index 62 is out of bounds for axis 0 with size 62] (if it is smaller than the specified height or width?)
                    if (x + 1 == len(heights)) or (y + 1 == len(widths)):
                        continue
                    else:
                        # Extract the new tile from the input raster using slice operation
                        new_tile = out_image[:, heights[x]:heights[x+1], widths[y]:widths[y+1]]
        
                        # skip tiles if percentage black pixels is higher than 10%
                        if np.count_nonzero((new_tile[0] == 0) & (new_tile[1] == 0) & (new_tile[2] == 0)) > 0.1*new_tile[0].size:
                            continue
                        else:
                        
                            # Calculate the transform for the new tile using rasterio.transform.from_origin() function: origin x (min) of the raster image + widths[y]*pixel_size, and y(max) - heights[x]*pixel_size
                            transform_new = rasterio.transform.from_origin(min_x + widths[y]*pixel_size, max_y - heights[x]*pixel_size, pixel_size, pixel_size)
                            
                            # Update the metadata information for the new tile
                            out_meta.update({"driver": "GTiff",
                                            "height": new_tile.shape[1],
                                            "width": new_tile.shape[2],
                                            "transform": transform_new})
                            
                            # Save the new tile to the output directory using rasterio.open() and dest.write() functions
                            with rasterio.open(os.path.join(output_dir, "{}_{}_{}_{}.tif".format(i, img_id, heights[x], widths[y])),
                                            "w", compress='lzw', **out_meta) as dest:
                                dest.write(new_tile)

In [9]:
def reproject(df_ds, current_crs, approximate_crs):
    """Reproject a geodataframe of a shapefile from one CRS to raster (target) CRS.

    Args:
        df_ds (DataFrame): A geodataframe of a shapefile layer.
        current_crs (str): The EPSG code of the current CRS of the shapefile.
        approximate_crs (str): The EPSG code of the target CRS.

    Returns:
        DataFrame: A geodataframe of a shapefile layer with the new coordinate reference system.
    """

    # Extract the geometries from the geodataframe
    geometries = df_ds['geometry']
    
    # Extract the coordinates of the geometries
    coords = pygeos.get_coordinates(geometries)
    
    # Set up a coordinate transformer to transform from the current CRS to the target CRS
    transformer = pyproj.Transformer.from_crs(current_crs, approximate_crs, always_xy=True)
    
    # Transform the coordinates to the target CRS
    new_coords = transformer.transform(coords[:, 0], coords[:, 1])
    
    # Update the geometries in the original geodataframe with the transformed coordinates
    return pygeos.set_coordinates(geometries.copy(), np.array(new_coords).T)

In [10]:
def split_annotations_to_geojson(df, tiles_path, geojson_folder):
    """
    Split annotations data into the raster tiles and convert to the format of train-net.py: 
    Pygeos to GPD to geojson.

    Args:
    df (GeoDataFrame): Vector dataset containing features to split.
    tiles_path (str): Path to the directory where the raster tiles are stored
    geojson_folder (str): Path to the directory where the output geojson files will be saved
    
    Returns:
    None
    """
    # Create spatial index for faster query
    #spatial index are the bounding boxes of the asset geometries
    spatial_index = pygeos.STRtree(df.geometry)
    id_obj = 0
    
    # Get a list of all files in the directory
    files = os.listdir(tiles_path)

    # Filter for only the files that end with ".tif"
    tif_tiles = [file for file in files if file.endswith(".tif")]

    # Iterate through each raster tile
    for tile in tqdm(tif_tiles):
        input_file = os.path.join(tiles_path,tile)
        with rasterio.open(input_file) as src:
            out_image = src.read()
            out_meta = src.meta
        # geom is the bounding box of the raster tile
        geom = pygeos.box(*src.bounds)
        
        # Query overlapping geometries from geom (tile) and the asset (from the spatial index)
        check_overlaps = spatial_index.query(geom, predicate='intersects')
        print (check_overlaps)

        # Create new geojson file for each overlapping geometry
        if len(check_overlaps) > 0:            
            get_matches = df.iloc[check_overlaps]
            get_exact_overlap = pygeos.intersection(get_matches['geometry'].values, geom)
            df_objs = pd.DataFrame()
            for polygon in get_exact_overlap:
                print (polygon)
                print (type(polygon))
                id_obj += 1
                #convert pygeos geometry to shapely geometry
                shapely_geom = pygeos.to_shapely(polygon)
                #geometries_objects = pygeos.to_wkt(object)
                df_row = pd.DataFrame()
                df_row['properties'] = [{"id":"{}".format(id_obj),"building":"yes"}]
                #convert shapely geometry to geopandas geometry
                df_row['geometry'] = gpd.GeoSeries([shapely_geom])
                #print (df_row)
                df_objs = df_objs.append(df_row)
            df_objs = df_objs.append(df_row)
            gdf_obj = gpd.GeoDataFrame(df_objs, geometry='geometry', crs="epsg:28992")
            gdf_obj.set_geometry(col='geometry', inplace=True)
                
            img_id = tile.split(".tif")[0]
            gdf_obj.to_file(os.path.join(geojson_folder, "{}.geojson".format(img_id)), driver="GeoJSON")
            

In [6]:
### Spliting a list of raster into several tiles of x width and y height skipping black tiles ###

#loop through the big tiles (rasters) and split them into smaller tiles
split_geotiff(raster_folder, tile_output_folder, width, height, nso_pix)

In [11]:
### Reproject the DataFrame of an annotation shapefile CRS to the target CRS as the raster tiles ###

df = pd.DataFrame(gpd.read_file(shp_path).copy())

# make df geometry column as a pygeos array
df.geometry = pygeos.from_shapely(df.geometry)

# reproject geometry column
df.geometry = reproject(df, annotations_crs, raster_crs)

/scistor/ivm/jpl204/miniconda3/envs/nso/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


In [ ]:
### Split annotations DataFrame into the new small tiles of width X and height Y & save it to geojson ###

split_annotations_to_geojson(df, tile_output_folder, json_folder)

  0%|          | 6/101873 [00:00<29:56, 56.69it/s]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


  0%|          | 19/101873 [00:00<32:06, 52.86it/s]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


  0%|          | 32/101873 [00:00<29:05, 58.35it/s]

...

 90%|█████████ | 91705/101873 [26:24<02:56, 57.64it/s]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


 90%|█████████ | 91713/101873 [26:24<02:49, 60.01it/s]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


 90%|█████████ | 91727/101873 [26:25<02:56, 57.55it/s]

In [ ]:
# Pyhtonic version to check if the geojsons are correct:
def split_annotations_to_geojson_pythonic (df, tiles_path, geojson_folder):
    """
    Split annotations data into the raster tiles and convert to the format of train-net.py: 
    Pygeos to GPD to geojson.

    Args:
    df (GeoDataFrame): Vector dataset containing features to split.
    tiles_path (str): Path to the directory where the raster tiles are stored
    geojson_folder (str): Path to the directory where the output geojson files will be saved
    
    Returns:
    None
    """
    # Create spatial index for faster query
    spatial_index = pygeos.STRtree(df.geometry)
    
    # Iterate through each raster tile
    for tile in os.listdir(tiles_path):
        if not tile.endswith('.tif'):
            continue
        
        input_file = os.path.join(tiles_path, tile)
        with rasterio.open(input_file) as src:
            geom = pygeos.box(*src.bounds)
            overlaps = spatial_index.query(geom, predicate='intersects')
        
        # Create new geojson file for each overlapping geometry
        if len(overlaps) > 0:
            matches = df.iloc[overlaps]
            exact_overlap = pygeos.intersection(matches.geometry.values, geom)
            
            objs = []
            for polygon in exact_overlap:
                shapely_geom = pygeos.to_shapely(polygon)
                objs.append({'properties': {'id': len(objs) + 1, 'building': 'yes'},
                             'geometry': shapely_geom})
            gdf = gpd.GeoDataFrame(objs, geometry='geometry', crs=src.crs)
            img_id = tile.split(".tif")[0]
            gdf.to_file(os.path.join(geojson_folder, f"{img_id}.geojson"), driver="GeoJSON")

In [ ]:
#delete_me
# #Version 14-03-23: does not work for several object in the same tile

# def split_annotations_to_geojson_x(df, tiles_path, geojson_folder):
#     """
#     Split annotations data into the raster tiles and convert to the format of train-net.py: 
#     Pygeos to GPD to geojson.

#     Args:
#     df (GeoDataFrame): Vector dataset containing features to split.
#     tiles_path (str): Path to the directory where the raster tiles are stored
#     geojson_folder (str): Path to the directory where the output geojson files will be saved
    
#     Returns:
#     None
#     """
#     # Create spatial index for faster query
#     spatial_index = pygeos.STRtree(df.geometry)
#     id_obj = 0
    
#         # Get a list of all files in the directory
#     files = os.listdir(tiles_path)

#     # Filter for only the files that end with ".tif"
#     tif_tiles = [file for file in files if file.endswith(".tif")]

#     # Iterate through each raster tile
#     for tile in tif_tiles:
#         input_file = os.path.join(tiles_path,tile)
#         with rasterio.open(input_file) as src:
#             out_image = src.read()
#             out_meta = src.meta
#         geom = pygeos.box(*src.bounds)
        
#         # Query overlapping geometries from the spatial index
#         check_overlaps = spatial_index.query(geom, predicate='intersects')
        
#         # Create new geojson file for each overlapping geometry
#         if len(check_overlaps) > 0:
#             id_obj += 1
#             get_matches = df.iloc[check_overlaps]
#             get_exact_overlap = pygeos.intersection(get_matches['geometry'].values, geom)
            
#             geometries_objects = pygeos.to_wkb(get_exact_overlap)
#             df_obj = pd.DataFrame()
#             df_obj['properties'] = [{"id":"{}".format(id_obj),"building":"yes"}]
#             df_obj['geometry'] = gpd.GeoSeries.from_wkb(geometries_objects)
#             gdf_obj = gpd.GeoDataFrame(df_obj, geometry='geometry', crs="epsg:28992")
#             gdf_obj.set_geometry(col='geometry', inplace=True)
            
#             img_id = tile.split(".tif")[0]
#             gdf_obj.to_file(os.path.join(geojson_folder, "{}.geojson".format(img_id)), driver="GeoJSON")

In [ ]:
### Delete me please ###

#adjust if more than one object:
###spliting annotations data into the raster tiles
###Adjustiong geojson to format of train-net.py: Pygeos to GPD to geojson

# path = "../gis/tiling/geojsons_annotations"

# spatial_index = pygeos.STRtree(df.geometry)
# id_obj = 0
# for tile in os.listdir(tiles_path):
#     input_file = os.path.join(tiles_path,tile)
#     with rasterio.open(input_file) as src:
#         out_image = src.read()#, out_transform = rasterio.mask.mask(src, shapes, crop=True)
#         out_meta = src.meta
#     geom = pygeos.box(*src.bounds)
#     check_overlaps = spatial_index.query(geom,predicate='intersects')
#     if len(check_overlaps) > 0:
#         id_obj += 1
#         get_matches = df.iloc[check_overlaps]
#         get_exact_overlap = pygeos.intersection(get_matches['geometry'].values,geom)
#         #old: get_exact_overlap = pygeos.intersection(row['geometry'].values,geom)
#     #for object_coord in get_exact_overlap:
#         # Pygeos coordinates to geoson coordinates to geopandas lead to backslash into the geosonfile so used to_wkb
#         geometries_objects = pygeos.to_wkb(get_exact_overlap)
#         df_obj = pd.DataFrame()
#         df_obj['properties'] = [{"id":"{}".format(id_obj),"building":"yes"}]
#         df_obj['geometry'] = gpd.GeoSeries.from_wkb(geometries_objects)
#         gdf_obj = gpd.GeoDataFrame(df_obj, geometry='geometry', crs="epsg:28992")
#         gdf_obj.set_geometry(col='geometry', inplace=True)
#         img_id = tile.split(".tif")[0]
#         gdf_obj.to_file(os.path.join(path,"{}.geojson".format(img_id)), driver="GeoJSON")

In [ ]:
### Delete me please ###
# Old Version without function

###spliting annotations data into the raster tiles
###Adjustiong geojson to format of train-net.py: Pygeos to GPD to geojson

# path = "../gis/tiling/geojsons_annotations"

# spatial_index = pygeos.STRtree(df.geometry)
# id_obj = 0
# for tile in os.listdir(tiles_path):
#     input_file = os.path.join(tiles_path,tile)
#     with rasterio.open(input_file) as src:
#         out_image = src.read()#, out_transform = rasterio.mask.mask(src, shapes, crop=True)
#         out_meta = src.meta
#     geom = pygeos.box(*src.bounds)
#     check_overlaps = spatial_index.query(geom,predicate='intersects')
#     if len(check_overlaps) > 0:
#         id_obj += 1
#         get_matches = df.iloc[check_overlaps]
#         get_exact_overlap = pygeos.intersection(get_matches.geometry.values,geom)
#         geometries_objects = pygeos.to_wkb(get_exact_overlap)
#         df_obj = pd.DataFrame()
#         df_obj['properties'] = [{"id":"{}".format(id_obj),"building":"yes"}]
#         df_obj['geometry'] = gpd.GeoSeries.from_wkb(geometries_objects)
#         gdf_obj = gpd.GeoDataFrame(df_obj, geometry='geometry', crs="epsg:28992")
#         gdf_obj.set_geometry(col='geometry', inplace=True)
#         img_id = tile.split(".tif")[0]
#         gdf_obj.to_file(os.path.join(out_path,"{}.geojson".format(img_id)), driver="GeoJSON")